In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests as r
import time

In [ ]:
def scrap_wikipedia_keyword(keyword:str)->str:
    str_keyword_html=keyword.replace(" ","%20")
    req=r.get(f"https://fr.wikipedia.org/wiki/{str_keyword_html}")
    soup = BeautifulSoup(req.text)
    list_phrases = [element.text for element in soup.find_all("p") if keyword in element.text.lower()]
    if list_phrases :
        return  list_phrases[0]
    return ""

In [ ]:
with open("../data/ingredients.csv","r") as f:
    list_ingredients=f.readlines()[1:]
    list_ingredients=list(map(lambda x: x.replace("\n",""),list_ingredients))

In [ ]:
list_phrase_exemple=list()
for ingredient in  list_ingredients:
    print(f"process {ingredient}")
    phrase=scrap_wikipedia_keyword(ingredient)
    list_phrase_exemple.append((ingredient,phrase))
    print(f"finish {ingredient} in {phrase}")
    time.sleep(10)
    